In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
import spacy
from textacy import preprocessing
import textacy
import nltk
import nltk, re, string
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
import pandas as pd 
import string
string.punctuation
import sklearn
from sklearn.metrics import pairwise_distances

In [3]:
review = pd.read_csv('/Users/yinzhihao/desktop/aud_2.csv')
review.head()

,name,review,date,rate,neg,neu,pos,compound
0,Alessandro d,Filme com narrativa arrastada e pouco imersiva...,9-Mar-22,1,0.000,1.000,0.000,0.0000
1,Benjamin P,One of the weakest MCU films in my opinion. Bu...,12-Jul-20,1,0.204,0.737,0.059,-0.8873
2,George M,"5 Bagger, it had Tim! Gregg was sorely missing...",10-Jul-20,1,0.403,0.597,0.000,-0.6588
3,Kenny L,"No es excelente, pero mantiene el espíritu de ...",9-Jul-20,1,0.196,0.804,0.000,-0.2960
4,Matheus A,"""Ant-Man and the Wasp"" is an average movie whe...",9-Jul-20,1,0.049,0.828,0.123,0.8430


In [4]:
data = review[['review', 'rate']]
data.head()

,review,rate
0,Filme com narrativa arrastada e pouco imersiva...,1
1,One of the weakest MCU films in my opinion. Bu...,1
2,"5 Bagger, it had Tim! Gregg was sorely missing...",1
3,"No es excelente, pero mantiene el espíritu de ...",1
4,"""Ant-Man and the Wasp"" is an average movie whe...",1


In [5]:
data = review['review']
docs = data.to_list()
print(docs)

['Filme com narrativa arrastada e pouco imersiva, efeitos não tão bons. O melhor do filme esta nas atuações.', "One of the weakest MCU films in my opinion. But does it mean the film is bad, or is it just a testament to the consistency of this franchise? I'd lean more towards the second option, because Ant-Man and the Wasp is entertaining as hell, even if it does trip at some points.", '5 Bagger, it had Tim! Gregg was sorely missing though :(', 'No es excelente, pero mantiene el espíritu de su antecesora.', '"Ant-Man and the Wasp" is an average movie where most of the jokes don\'t work like in the Ant-Man solo film, Villain ??? The film does not develop its antagonist well. The duo of protagonists shines in their respective roles. Now let\'s go to the grand post-credit scene, so here comes Spoiler. In the scene, we see Hank Pim, the Wasp and Pim\'s wife being wiped out by the "snap" while Scott Lang is trapped in the quantum realm, where it is reflected in the main events of "Avengers: 

In [6]:
stop_words = stopwords.words('english')
def get_doc_tokens(doc):
    tokens=[token.strip() \
            for token in nltk.word_tokenize(doc.lower()) \
            if token.strip() not in stop_words and\
               token.strip() not in string.punctuation]
    
    # you can add bigrams, collocations, or lemmatization here
    
    # create token count dictionary
    token_count=nltk.FreqDist(tokens)
    
    # or you can create dictionary by yourself
    #token_count={token:tokens.count(token) for token in set(tokens)}
    return token_count

# step 2. process all documents to 
# a dictionary of dictionaries
docs_tokens={idx:get_doc_tokens(doc) \
             for idx,doc in enumerate(docs)}
docs_tokens

{0: FreqDist({'filme': 2, 'com': 1, 'narrativa': 1, 'arrastada': 1, 'e': 1, 'pouco': 1, 'imersiva': 1, 'efeitos': 1, 'não': 1, 'tão': 1, ...}),
 1: FreqDist({'one': 1, 'weakest': 1, 'mcu': 1, 'films': 1, 'opinion': 1, 'mean': 1, 'film': 1, 'bad': 1, 'testament': 1, 'consistency': 1, ...}),
 2: FreqDist({'5': 1, 'bagger': 1, 'tim': 1, 'gregg': 1, 'sorely': 1, 'missing': 1, 'though': 1}),
 3: FreqDist({'es': 1, 'excelente': 1, 'pero': 1, 'mantiene': 1, 'el': 1, 'espíritu': 1, 'de': 1, 'su': 1, 'antecesora': 1}),
 4: FreqDist({'``': 4, "''": 4, 'scene': 4, 'ant-man': 3, 'wasp': 3, "n't": 2, 'film': 2, "'s": 2, 'pim': 2, 'average': 1, ...}),
 5: FreqDist({'captures': 1, 'tone': 1, 'first': 1, 'movie': 1, 'fun': 1, 'funny': 1, 'ride': 1}),
 6: FreqDist({'fun': 1, 'fast': 1, 'paced': 1, 'addition': 1, 'marvel': 1, 'film': 1, 'canon': 1, 'invested': 1, 'stakes': 1, 'without': 1, ...}),
 7: FreqDist({'film': 2, 'thought': 1, 'okay': 1, 'pretty': 1, 'disappointing': 1, 'first': 1, '``': 1, 'ant

In [7]:
dtm=pd.DataFrame.from_dict(docs_tokens, orient="index" )
dtm
dtm=dtm.fillna(0)
dtm

# sort by index (i.e. doc id)
dtm = dtm.sort_index(axis = 0)
dtm

,filme,com,narrativa,arrastada,e,pouco,imersiva,efeitos,não,tão,...,arrives,unimpressive,rotten,disgusting,convenient,things,hold,gives,stone,disney+
0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,filme,com,narrativa,arrastada,e,pouco,imersiva,efeitos,não,tão,...,arrives,unimpressive,rotten,disgusting,convenient,things,hold,gives,stone,disney+
0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,filme,com,narrativa,arrastada,e,pouco,imersiva,efeitos,não,tão,...,arrives,unimpressive,rotten,disgusting,convenient,things,hold,gives,stone,disney+
0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
142,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
tf=dtm.values

# sum the value of each row
doc_len=tf.sum(axis=1)
doc_len

# divide dtm matrix by the doc length matrix
tf=np.divide(tf, doc_len[:,None])

# set float precision to print nicely
np.set_printoptions(precision=2)

tf

array([ 16.,  23.,   7.,   9.,  67.,   7.,  15.,  14.,  21.,  12.,  15.,
        16.,  42.,   6.,  11.,  16.,  16.,  23.,  23.,  10.,  46.,  38.,
        14.,  28.,  13.,   5.,   6., 282.,   4.,  50.,  11.,  42.,  24.,
         2.,  62.,  29.,  49.,  67.,   7.,   5., 133.,  10.,  16.,  36.,
        52.,   9., 326.,   3.,  16., 129.,  50.,  12.,  14.,  15.,  20.,
        22.,  34.,   7.,  35.,   9.,  16.,   7.,  33.,   4.,  18.,  25.,
        13.,   3.,  20.,   3.,  11.,  24.,  19.,  74.,  31., 192.,   3.,
       312.,  20.,  10.,  17.,  22.,  19.,  26.,   5.,  36.,   9.,  10.,
        25.,   4.,  21.,   8.,   4.,  19.,  17.,  19.,  23.,  14.,  12.,
        39.,  41.,  31.,  40.,  13.,   9.,   8.,  26.,   5.,  22.,  10.,
        38.,   5.,  18.,  72.,   5.,   4.,   4.,   4.,   3.,  14.,  20.,
       556.,  65.,   9.,   2.,   6.,  24.,  10.,  33.,   8., 333.,  13.,
        11.,   8.,  14.,  41.,  17.,  50.,  11.,  42.,  24.,   2.,  62.,
        29.,  49.,   5.])

array([[0.12, 0.06, 0.06, ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       ...,
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.2 ]])

In [9]:
df=np.where(tf>0,1,0)
df

# get idf
idf=np.log(np.divide(len(docs), \
        np.sum(df, axis=0)))+1
print("\nIDF Matrix")
idf

# what is the size of idf array?

smoothed_idf=np.log(np.divide(len(docs)+1, np.sum(df, axis=0)+1))+1
print("\nSmoothed IDF Matrix")
smoothed_idf

array([[1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]])


IDF Matrix


array([5.29, 5.29, 5.98, ..., 5.98, 5.98, 5.98])


Smoothed IDF Matrix


array([4.89, 4.89, 5.3 , ..., 5.3 , 5.3 , 5.3 ])

In [10]:
print("TF-IDF Matrix")
s = tf*idf
s

tf_idf=normalize(tf*idf)   # is broadcast possible here?
tf_idf

print("\nSmoothed TF-IDF Matrix")
smoothed_tf_idf=normalize(tf*smoothed_idf)
smoothed_tf_idf

TF-IDF Matrix


array([[0.66, 0.33, 0.37, ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       ...,
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 1.2 ]])

array([[0.43, 0.22, 0.24, ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       ...,
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.63]])


Smoothed TF-IDF Matrix


array([[0.44, 0.22, 0.24, ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       ...,
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.59]])

In [11]:
pd.options.display.float_format = '{:,.2f}'.format # set format for float

pd.DataFrame(smoothed_tf_idf, columns = dtm.columns)

,filme,com,narrativa,arrastada,e,pouco,imersiva,efeitos,não,tão,...,arrives,unimpressive,rotten,disgusting,convenient,things,hold,gives,stone,disney+
0,0.44,0.22,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
142,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
143,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
144,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [12]:
top=smoothed_tf_idf.argsort()[:,::-1][:,0:3]
top

for row in top:
    print([dtm.columns[x] for x in row])

array([[   0,    7,   14],
       [  36,   30,   27],
       [  38,   39,   40],
       [  50,   48,   47],
       [  77,   32,   55],
       [ 101,  106,  102],
       [ 111,  119,  112],
       [  21,  124,   31],
       [ 128,  127,  135],
       [ 145,  144,  148],
       [ 155,  160,  159],
       [ 171,  170,  168],
       [ 166,  175,  199],
       [ 205,  204,  203],
       [ 211,  208,  206],
       [ 157,  219,  217],
       [  59,  225,  224],
       [ 231,  233,  236],
       [ 246,   58,  250],
       [ 260,  258,  255],
       [ 271,  267,  266],
       [ 300,  305,  301],
       [ 319,  316,  321],
       [ 335,  330,  329],
       [ 339,  177,  338],
       [ 343,  341,  342],
       [ 344,  346,  305],
       [  21,  358,  357],
       [ 470,  469,  471],
       [ 141,   57,  478],
       [ 502,  501,  137],
       [ 504,  276,  163],
       [ 523,  521,  527],
       [ 411,   57, 1915],
       [  73,  234,  556],
       [ 130,  103,  234],
       [ 585,   57,  240],
 

['filme', 'efeitos', 'atuações']
['trip', 'option', 'lean']
['5', 'bagger', 'tim']
['espíritu', 'mantiene', 'pero']
['scene', 'wasp', "''"]
['captures', 'ride', 'tone']
['canon', 'laser', 'invested']
['film', '2015', 'ant-man']
['negligible', 'reasonably', 'consequential']
['problem', "'got", 'solved']
['quicksilver', 'wanda', 'introduction']
['hardly', 'uneven', 'disappoints']
['ultron', 'bogged', '2-and-a-half']
['greatest', '2nd', 'screw']
['period', 'wandavision', 'yeah']
['...', 'hulk', 'ironman']
["n't", 'banner/widow', 'sake']
['many', 'treatment', 'nowhere']
['joke', 'jokes', 'death']
['8.5/10', 'audibly', 'anyone']
['hero', 'fight', 'last']
['make', 'wakanda', 'sense']
['leveraged', 'top', 'situation']
['fantastic', 'moral', 'importantly']
['appeal', 'story', 'personally']
['worldbuilding', '17/25', 'moment']
['forever', '3,000', 'wakanda']
['film', 'needed', 'felt']
['disappointed', 'immensely', 'hype']
['great', 'movie', 'shine']
['helps', 'installment', 'infinity']
['pratt'